# 21 - Apache Flink vs Spark Streaming

Porownanie dwoch najpopularniejszych frameworkow do przetwarzania strumieniowego w ekosystemie Big Data.

**Tematy:**
- Architektura Flink: JobManager, TaskManager, Slots
- Porownanie: Flink DataStream API vs Spark Structured Streaming
- Flink w Python (PyFlink): Table API i DataStream API
- Event time vs processing time
- Watermarks i late data handling
- Exactly-once semantics: Flink checkpointing vs Spark WAL
- State management: Flink (RocksDB) vs Spark (in-memory/HDFS)
- Windowing: tumbling, sliding, session windows
- Benchmark: latency Flink vs Spark na strumieniu ratings
- Kiedy wybrac Flink, a kiedy Spark Streaming?

## 1. Architektura Apache Flink

Flink to framework do **natywnego przetwarzania strumieniowego** (stream-first). W przeciwienstwie do Sparka, ktory traktuje strumien jako serie micro-batchy, Flink przetwarza kazde zdarzenie indywidualnie.

```
                    ┌──────────────────────┐
                    │   Flink Client        │
                    │   (submit job JAR)    │
                    └──────────┬───────────┘
                               │
                    ┌──────────▼───────────┐
                    │    JobManager         │  <- koordynator klastra
                    │  ┌─────────────────┐  │
                    │  │   Dispatcher    │  │  <- przyjmuje joby
                    │  │   ResourceMgr   │  │  <- zarzadza zasobami
                    │  │   JobMaster     │  │  <- wykonuje DAG jednego joba
                    │  └─────────────────┘  │
                    └──────────┬───────────┘
                               │
            ┌──────────────────┼──────────────────┐
            │                  │                   │
   ┌────────▼────────┐ ┌──────▼────────┐ ┌───────▼───────┐
   │  TaskManager 1  │ │ TaskManager 2 │ │ TaskManager 3 │
   │                 │ │               │ │               │
   │ ┌─────┐┌─────┐ │ │ ┌─────┐┌─────┐│ │ ┌─────┐┌─────┐│
   │ │Slot1││Slot2│ │ │ │Slot1││Slot2││ │ │Slot1││Slot2││
   │ │Task ││Task │ │ │ │Task ││Task ││ │ │Task ││Task ││
   │ └─────┘└─────┘ │ │ └─────┘└─────┘│ │ └─────┘└─────┘│
   └─────────────────┘ └──────────────┘ └───────────────┘
```

### Kluczowe komponenty:

| Komponent | Rola | Odpowiednik w Spark |
|-----------|------|--------------------|
| **JobManager** | Koordynacja, scheduling, checkpointing | Driver |
| **TaskManager** | Wykonywanie taskow (worker) | Executor |
| **Slot** | Jednostka zasobow w TaskManager | Core/Thread w Executor |
| **JobGraph** | DAG operatorow | DAG stages |
| **Checkpoint** | Snapshot stanu do odtworzenia | Checkpoint (WAL/state store) |

### Flink vs Spark - roznice architektoniczne:

| Cecha | Flink | Spark Structured Streaming |
|-------|-------|---------------------------|
| **Model** | Natywny streaming (event-by-event) | Micro-batch (domyslnie) |
| **Latencja** | Milisekundy | Sekundy (micro-batch interval) |
| **Stan** | Wbudowany (RocksDB / heap) | Zewnetrzny (HDFS state store) |
| **Checkpointing** | Asynchroniczne barierowe | Synchroniczne per micro-batch |
| **Batch** | Streaming jako special case | Natywny batch engine |
| **API** | DataStream, Table/SQL | DataFrame/Dataset, SQL |

## 2. Setup - Spark Structured Streaming

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
import json
import os
import random
from datetime import datetime, timedelta

spark = SparkSession.builder \
    .appName("21_Flink_vs_Spark_Streaming") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
jdbc_props = {"user": "recommender", "password": "recommender", "driver": "org.postgresql.Driver"}

# Katalogi na dane strumieniowe
STREAM_DIR = "/tmp/flink_vs_spark_stream"
CHECKPOINT_DIR = "/tmp/flink_vs_spark_ckpt"
os.makedirs(STREAM_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Wyczysc stare dane
for f in os.listdir(STREAM_DIR):
    os.remove(os.path.join(STREAM_DIR, f))

print(f"Spark UI: {spark.sparkContext.uiWebUrl}")
print(f"Stream dir: {STREAM_DIR}")

## 3. Setup - PyFlink (Table API)

PyFlink to oficjalny Python API dla Apache Flink. Oferuje dwa glowne interfejsy:
- **Table API** - deklaratywny, relacyjny (podobny do DataFrame)
- **DataStream API** - niski poziom, pelna kontrola nad przetwarzaniem

W naszym srodowisku Flink JobManager jest dostepny pod adresem `flink-jobmanager:8081`.

In [ ]:
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.expressions import col as flink_col, lit as flink_lit
from pyflink.table.window import Tumble, Slide, Session
from pyflink.common import Row
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment

# Flink Table Environment - tryb strumieniowy
env_settings = EnvironmentSettings.in_streaming_mode()
t_env = TableEnvironment.create(env_settings)

# Konfiguracja Flink
t_env.get_config().set("parallelism.default", "2")
t_env.get_config().set("pipeline.jars", 
    "file:///opt/flink/lib/flink-connector-jdbc-3.1.0-1.17.jar;"
    "file:///opt/flink/lib/postgresql-42.7.1.jar")

print("PyFlink Table Environment utworzony (streaming mode)")
print(f"Flink JobManager UI: http://flink-jobmanager:8081")

## 4. Event Time vs Processing Time

Kluczowa koncepcja w przetwarzaniu strumieniowym - jaki czas przypisac zdarzeniu?

```
Zdarzenie: Uzytkownik ocenia film o 14:00:00
Siec opoznia o 3 sekundy
System odbiera o 14:00:03

Event time:      14:00:00  (kiedy zdarzenie WYSTAPILO)
Processing time: 14:00:03  (kiedy system je ODEBRALO)
Ingestion time:  14:00:03  (kiedy weszlo do systemu)
```

| Cecha | Event Time | Processing Time |
|-------|------------|----------------|
| **Determinizm** | Tak (powtarzalne wyniki) | Nie (zalezne od szybkosci) |
| **Opoznione dane** | Obsluguje (watermarks) | Ignoruje problem |
| **Zlozonosc** | Wyzsza (wymaga watermarks) | Prostsza |
| **Uzycie** | Analityka, billing, audyt | Monitoring, alerty |

### Flink:
- Event time jest **domyslnym** trybem od Flink 1.12+
- Watermarks definiowane per source

### Spark Structured Streaming:
- Event time obslugiwany przez kolumne timestamp w DataFrame
- Watermarks definiowane przez `.withWatermark()`

In [ ]:
# --- Generator danych strumieniowych (wspolny dla Spark i Flink) ---

def generate_rating_events(batch_id, n=100, late_fraction=0.1):
    """Generuj batch ocen z czescia opoznionych zdarzen (late events)."""
    ratings = []
    base_time = datetime.now()
    
    for i in range(n):
        # Niektore zdarzenia sa "opoznione" - event_time znacznie wczesniejszy
        if random.random() < late_fraction:
            event_time = base_time - timedelta(seconds=random.randint(30, 120))
            is_late = True
        else:
            event_time = base_time - timedelta(seconds=random.randint(0, 5))
            is_late = False
        
        ratings.append({
            "user_id": random.randint(1, 1000),
            "movie_id": random.choice([1, 2, 50, 110, 260, 296, 318, 356, 480, 527]),
            "rating": random.choice([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]),
            "event_time": event_time.strftime("%Y-%m-%d %H:%M:%S"),
            "is_late": is_late
        })
    
    path = os.path.join(STREAM_DIR, f"batch_{batch_id}.json")
    with open(path, 'w') as f:
        for r in ratings:
            f.write(json.dumps(r) + '\n')
    return path

# Wygeneruj poczatkowe dane
for i in range(3):
    generate_rating_events(i, n=100, late_fraction=0.15)

print(f"Wygenerowano 3 batche po 100 zdarzen (15% opoznionych)")
print(f"Przykladowe zdarzenie:")
with open(os.path.join(STREAM_DIR, "batch_0.json")) as f:
    print(json.loads(f.readline()))

## 5. Watermarks i Late Data Handling

Watermark to mechanizm informujacy system: "wszystkie zdarzenia z czasem <= W juz dotarly".

```
Strumien zdarzen (event time):
  [14:00:01] [14:00:03] [14:00:02] [14:00:05] [13:59:50] [14:00:06]
                                                ^^^^^^^^^
                                                opoznione o 16s!

Watermark z tolerancja 10s:
  Max event time = 14:00:06
  Watermark = 14:00:06 - 10s = 13:59:56
  
  Zdarzenie 13:59:50 < 13:59:56 → ODRZUCONE (za pozno!)
  Gdyby tolerancja byla 20s → Watermark = 13:59:46 → ZAAKCEPTOWANE
```

### Porownanie Watermarks:

| Cecha | Flink | Spark Structured Streaming |
|-------|-------|---------------------------|
| **Definicja** | `WatermarkStrategy.forBoundedOutOfOrderness()` | `.withWatermark("col", "delay")` |
| **Granularnosc** | Per event (rekord po rekordzie) | Per micro-batch |
| **Custom logic** | Pelna kontrola (`WatermarkGenerator`) | Tylko opoznienie stalej dlugosci |
| **Late data** | Side output (oddzielny strumien) | Odrzucenie (drop) |

In [ ]:
# --- SPARK: Watermarks i obsluga opoznionych danych ---

import shutil
for d in os.listdir(CHECKPOINT_DIR):
    shutil.rmtree(os.path.join(CHECKPOINT_DIR, d), ignore_errors=True)

rating_schema = StructType([
    StructField("user_id", IntegerType()),
    StructField("movie_id", IntegerType()),
    StructField("rating", DoubleType()),
    StructField("event_time", StringType()),
    StructField("is_late", BooleanType())
])

spark_stream = spark.readStream \
    .format("json") \
    .schema(rating_schema) \
    .option("maxFilesPerTrigger", 1) \
    .load(STREAM_DIR) \
    .withColumn("event_ts", to_timestamp("event_time"))

# Watermark: akceptuj opoznienia do 30 sekund
windowed_spark = spark_stream \
    .withWatermark("event_ts", "30 seconds") \
    .groupBy(
        window(col("event_ts"), "15 seconds"),
        col("movie_id")
    ) \
    .agg(
        count("*").alias("cnt"),
        round(avg("rating"), 2).alias("avg_rating")
    )

query_spark_wm = windowed_spark.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("spark_watermark_demo") \
    .option("checkpointLocation", f"{CHECKPOINT_DIR}/spark_wm") \
    .start()

time.sleep(5)

spark.sql("""
    SELECT window.start, window.end, movie_id, cnt, avg_rating
    FROM spark_watermark_demo
    ORDER BY window.start DESC, cnt DESC
    LIMIT 15
""").show(truncate=False)

query_spark_wm.stop()
print("Spark: zdarzenia opoznione >30s sa odrzucane (brak side output)")

In [ ]:
# --- FLINK: Watermarks z Table API ---
# Flink oferuje side output dla opoznionych danych (niedostepne w Spark)

# Definicja tabeli zrodlowej z watermarkiem w Flink SQL
t_env.execute_sql("""
    CREATE TEMPORARY TABLE rating_stream (
        user_id INT,
        movie_id INT,
        rating DOUBLE,
        event_time TIMESTAMP(3),
        is_late BOOLEAN,
        WATERMARK FOR event_time AS event_time - INTERVAL '30' SECOND
    ) WITH (
        'connector' = 'filesystem',
        'path' = '/tmp/flink_vs_spark_stream',
        'format' = 'json'
    )
""")

# Flink: okno tumbling 15s z watermarkiem
result = t_env.execute_sql("""
    SELECT 
        TUMBLE_START(event_time, INTERVAL '15' SECOND) AS window_start,
        TUMBLE_END(event_time, INTERVAL '15' SECOND) AS window_end,
        movie_id,
        COUNT(*) AS cnt,
        ROUND(AVG(rating), 2) AS avg_rating
    FROM rating_stream
    GROUP BY 
        TUMBLE(event_time, INTERVAL '15' SECOND),
        movie_id
""")

# Wyswietl wyniki (Flink wykonuje job na JobManager)
print("Flink: wyniki okna tumbling 15s z watermark 30s")
print("Flink moze dodatkowo wyslac opoznione dane do side output!")
with result.collect() as results:
    for i, row in enumerate(results):
        if i >= 15:
            break
        print(row)

## 6. Exactly-Once Semantics i State Management

### Gwarancje dostarczenia:

```
At-most-once:   Zdarzenie moze zostac utracone        (najszybsze)
At-least-once:  Zdarzenie przetworzone >= 1 raz       (mozliwe duplikaty)
Exactly-once:   Zdarzenie przetworzone dokladnie 1 raz (najtrudniejsze)
```

### Flink Checkpointing (Chandy-Lamport):
```
Source ──► Op1 ──► Op2 ──► Sink
  │         │        │       │
  ▼         ▼        ▼       ▼
 [barrier] przeplyw barierowy
  │         │        │       │
  ▼         ▼        ▼       ▼
 State     State    State   Commit
 snapshot  snapshot snapshot offset
  │         │        │       │
  └─────────┴────────┴───────┘
              │
        Distributed Snapshot
        (RocksDB / Heap → HDFS/S3)
```

### Spark Checkpoint + WAL:
```
Micro-batch N:
  1. Zapisz offsety do WAL (Write-Ahead Log)
  2. Przetworzenie batcha
  3. Zapisz state do HDFS state store
  4. Commit offsetow

Awaria → odtworzenie z ostatniego checkpointu + replay WAL
```

### State Management - porownanie:

| Cecha | Flink | Spark Structured Streaming |
|-------|-------|---------------------------|
| **Backend** | RocksDB (dysk) lub Heap (RAM) | HDFS State Store (domyslny) |
| **Skalowalnosc** | TB stanu (RocksDB) | Ograniczona pamiec executora |
| **Inkrementalnosc** | Incremental checkpoints | Pelny snapshot co batch |
| **Queryable state** | Tak (eksperymentalne) | Nie |
| **TTL stanu** | Wbudowany State TTL | Recznie (mapGroupsWithState) |

In [ ]:
# --- FLINK: Konfiguracja checkpointingu ---

# W PyFlink konfigurujemy checkpointing przez StreamExecutionEnvironment
stream_env = StreamExecutionEnvironment.get_execution_environment()

# Wlacz checkpointing co 10 sekund
stream_env.enable_checkpointing(10000)  # 10s interval

# Konfiguracja checkpoint
from pyflink.datastream.checkpointing_mode import CheckpointingMode
stream_env.get_checkpoint_config().set_checkpointing_mode(CheckpointingMode.EXACTLY_ONCE)
stream_env.get_checkpoint_config().set_min_pause_between_checkpoints(5000)  # min 5s miedzy ckpt
stream_env.get_checkpoint_config().set_checkpoint_timeout(60000)  # timeout 60s
stream_env.get_checkpoint_config().set_max_concurrent_checkpoints(1)

# State backend - RocksDB dla duzych stanow
# stream_env.set_state_backend(RocksDBStateBackend("hdfs://namenode:9000/flink/checkpoints"))

print("Flink checkpointing skonfigurowany:")
print(f"  Mode: EXACTLY_ONCE")
print(f"  Interval: 10s")
print(f"  Min pause: 5s")
print(f"  Timeout: 60s")
print(f"  Max concurrent: 1")

# Dla porownania - Spark checkpointing konfigurujemy per query:
print("\nSpark checkpointing (per query):")
print('  .option("checkpointLocation", "hdfs://namenode:9000/spark/checkpoints/query_name")')
print("  Spark automatycznie zapisuje state + offsets co micro-batch")

## 7. Windowing - porownanie implementacji

Trzy glowne typy okien czasowych:

```
TUMBLING (stale, nieprzekrywajace sie):
  |---W1---|---W2---|---W3---|---W4---|
  0       10      20      30      40   (sekundy)

SLIDING (przesuwane, przekrywajace sie):
  |------W1------|
       |------W2------|
            |------W3------|
  Okno=15s, Slide=5s

SESSION (dynamiczne, oparte na aktywnosci):
  |--W1--|    gap    |----W2----|   gap   |--W3--|
  zdarzenia   >10s   zdarzenia   >10s    zdarzenia
  (gap = session timeout)
```

### Session Windows - kluczowa roznica!

| Cecha | Flink | Spark |
|-------|-------|-------|
| **Tumbling** | `TUMBLE(time, INTERVAL)` | `window(col, "duration")` |
| **Sliding** | `HOP(time, slide, size)` | `window(col, "size", "slide")` |
| **Session** | `SESSION(time, gap)` - natywne! | Brak natywnej obslugi! |

Flink obsluguje **session windows** natywnie, co jest ogromna zaleta przy analizie sesji uzytkownikow. Spark wymaga obejscia przez `mapGroupsWithState`.

In [ ]:
# --- SPARK: Tumbling i Sliding windows ---

for d in os.listdir(CHECKPOINT_DIR):
    shutil.rmtree(os.path.join(CHECKPOINT_DIR, d), ignore_errors=True)

# Wygeneruj wiecej danych
for i in range(10, 16):
    generate_rating_events(i, n=80)

spark_stream2 = spark.readStream \
    .format("json") \
    .schema(rating_schema) \
    .option("maxFilesPerTrigger", 2) \
    .load(STREAM_DIR) \
    .withColumn("event_ts", to_timestamp("event_time"))

# Tumbling window - 15s
tumbling_spark = spark_stream2 \
    .withWatermark("event_ts", "30 seconds") \
    .groupBy(window(col("event_ts"), "15 seconds")) \
    .agg(
        count("*").alias("total_ratings"),
        countDistinct("user_id").alias("unique_users"),
        round(avg("rating"), 2).alias("avg_rating")
    )

q_tumbling = tumbling_spark.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("spark_tumbling") \
    .option("checkpointLocation", f"{CHECKPOINT_DIR}/tumbling") \
    .start()

time.sleep(5)
print("=== SPARK: Tumbling Window 15s ===")
spark.sql("""
    SELECT window.start, window.end, total_ratings, unique_users, avg_rating
    FROM spark_tumbling ORDER BY window.start DESC LIMIT 10
""").show(truncate=False)

# Sliding window - okno 30s, slide 10s
spark_stream3 = spark.readStream \
    .format("json") \
    .schema(rating_schema) \
    .option("maxFilesPerTrigger", 2) \
    .load(STREAM_DIR) \
    .withColumn("event_ts", to_timestamp("event_time"))

sliding_spark = spark_stream3 \
    .withWatermark("event_ts", "30 seconds") \
    .groupBy(window(col("event_ts"), "30 seconds", "10 seconds")) \
    .agg(count("*").alias("total"), round(avg("rating"), 2).alias("avg"))

q_sliding = sliding_spark.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("spark_sliding") \
    .option("checkpointLocation", f"{CHECKPOINT_DIR}/sliding") \
    .start()

time.sleep(5)
print("\n=== SPARK: Sliding Window 30s / slide 10s ===")
spark.sql("""
    SELECT window.start, window.end, total, avg
    FROM spark_sliding ORDER BY window.start DESC LIMIT 10
""").show(truncate=False)

q_tumbling.stop()
q_sliding.stop()

In [ ]:
# --- FLINK: Tumbling, Sliding, i Session Windows ---

# Flink SQL: Tumbling window
print("=== FLINK: Tumbling Window 15s ===")
result_tumble = t_env.execute_sql("""
    SELECT
        TUMBLE_START(event_time, INTERVAL '15' SECOND) AS w_start,
        TUMBLE_END(event_time, INTERVAL '15' SECOND) AS w_end,
        COUNT(*) AS total_ratings,
        COUNT(DISTINCT user_id) AS unique_users,
        ROUND(AVG(rating), 2) AS avg_rating
    FROM rating_stream
    GROUP BY TUMBLE(event_time, INTERVAL '15' SECOND)
""")
with result_tumble.collect() as results:
    for i, row in enumerate(results):
        if i >= 10:
            break
        print(row)

# Flink SQL: Sliding (HOP) window
print("\n=== FLINK: Sliding (HOP) Window 30s / slide 10s ===")
result_hop = t_env.execute_sql("""
    SELECT
        HOP_START(event_time, INTERVAL '10' SECOND, INTERVAL '30' SECOND) AS w_start,
        HOP_END(event_time, INTERVAL '10' SECOND, INTERVAL '30' SECOND) AS w_end,
        COUNT(*) AS total,
        ROUND(AVG(rating), 2) AS avg
    FROM rating_stream
    GROUP BY HOP(event_time, INTERVAL '10' SECOND, INTERVAL '30' SECOND)
""")
with result_hop.collect() as results:
    for i, row in enumerate(results):
        if i >= 10:
            break
        print(row)

# Flink SQL: Session window - NIEDOSTEPNE w Spark!
print("\n=== FLINK: Session Window (gap=20s) - unikalna funkcja Flink! ===")
result_session = t_env.execute_sql("""
    SELECT
        SESSION_START(event_time, INTERVAL '20' SECOND) AS session_start,
        SESSION_END(event_time, INTERVAL '20' SECOND) AS session_end,
        user_id,
        COUNT(*) AS ratings_in_session,
        ROUND(AVG(rating), 2) AS avg_rating
    FROM rating_stream
    GROUP BY SESSION(event_time, INTERVAL '20' SECOND), user_id
""")
with result_session.collect() as results:
    for i, row in enumerate(results):
        if i >= 10:
            break
        print(row)

## 8. Benchmark: Latency - Flink vs Spark

Kluczowa roznica miedzy Flink a Spark Streaming to **latencja**:
- **Flink**: przetwarza zdarzenie natychmiast po przyjsciu (ms)
- **Spark**: czeka na zebranie micro-batcha, nastepnie przetwarza (sekundy)

Zmierzymy czas od wygenerowania zdarzenia do pojawienia sie wyniku.

In [ ]:
# --- Benchmark latencji: Spark Structured Streaming ---

import shutil
# Wyczysc katalogi
for f in os.listdir(STREAM_DIR):
    os.remove(os.path.join(STREAM_DIR, f))
for d in os.listdir(CHECKPOINT_DIR):
    shutil.rmtree(os.path.join(CHECKPOINT_DIR, d), ignore_errors=True)

spark_stream_bench = spark.readStream \
    .format("json") \
    .schema(rating_schema) \
    .option("maxFilesPerTrigger", 1) \
    .load(STREAM_DIR) \
    .withColumn("event_ts", to_timestamp("event_time"))

bench_query = spark_stream_bench \
    .groupBy("movie_id") \
    .agg(count("*").alias("cnt"), round(avg("rating"), 2).alias("avg")) \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("spark_bench") \
    .option("checkpointLocation", f"{CHECKPOINT_DIR}/bench") \
    .start()

# Mierz czas od wstawienia danych do pojawienia sie wynikow
spark_latencies = []
for i in range(5):
    t_start = time.time()
    generate_rating_events(100 + i, n=50)
    
    # Czekaj na przetworzenie
    prev_count = 0
    while True:
        time.sleep(0.1)
        current = spark.sql("SELECT SUM(cnt) as total FROM spark_bench").collect()[0][0]
        if current and current > prev_count:
            latency = time.time() - t_start
            spark_latencies.append(latency)
            prev_count = current
            break
        if time.time() - t_start > 30:
            spark_latencies.append(30.0)
            break

bench_query.stop()

print("=== Spark Structured Streaming - Latency ===")
print(f"Latencje per batch: {[f'{l:.2f}s' for l in spark_latencies]}")
print(f"Srednia latencja: {sum(spark_latencies)/len(spark_latencies):.2f}s")
print(f"Min latencja: {min(spark_latencies):.2f}s")
print(f"\nSpark przetwarza w micro-batchach, wiec latencja = trigger interval + processing")

In [ ]:
# --- Benchmark latencji: PyFlink ---
# Flink przetwarza event-by-event, wiec oczekujemy nizszej latencji

# Mierzymy czas wykonania zapytania Flink SQL na tych samych danych
flink_latencies = []

for i in range(5):
    t_start = time.time()
    
    result = t_env.execute_sql("""
        SELECT movie_id, COUNT(*) AS cnt, ROUND(AVG(rating), 2) AS avg_rating
        FROM rating_stream
        GROUP BY movie_id
    """)
    
    # Odczytaj pierwszy wynik
    with result.collect() as results:
        first_row = next(iter(results), None)
    
    latency = time.time() - t_start
    flink_latencies.append(latency)

print("=== PyFlink - Latency ===")
print(f"Latencje per run: {[f'{l:.2f}s' for l in flink_latencies]}")
print(f"Srednia latencja: {sum(flink_latencies)/len(flink_latencies):.2f}s")
print(f"Min latencja: {min(flink_latencies):.2f}s")

print("\n=== Porownanie ===")
print(f"Spark srednia: {sum(spark_latencies)/len(spark_latencies):.2f}s")
print(f"Flink srednia: {sum(flink_latencies)/len(flink_latencies):.2f}s")
print("\nUWAGA: W produkcyjnym klastrze roznica bylaby jeszcze wieksza.")
print("Flink osiaga latencje <100ms, Spark typowo 500ms-2s (micro-batch).")

## 9. Kiedy wybrac Flink, a kiedy Spark Streaming?

### Wybierz **Apache Flink** gdy:

| Scenariusz | Dlaczego Flink? |
|------------|----------------|
| Ultra-niska latencja (<100ms) | Natywny streaming, event-by-event |
| Skomplikowany stan (TB) | RocksDB backend, incremental checkpoints |
| Session windows | Natywna obsluga, brak w Spark |
| Complex Event Processing (CEP) | Wbudowana biblioteka Flink CEP |
| Opoznione dane wymagaja obslugi | Side output dla late events |
| Event time jest krytyczny | Domyslny tryb, zaawansowane watermarks |

### Wybierz **Spark Structured Streaming** gdy:

| Scenariusz | Dlaczego Spark? |
|------------|----------------|
| Juz uzywasz Spark do batch | Ten sam kod batch i streaming! |
| Latencja ~1s jest akceptowalna | Micro-batch wystarczy |
| Potrzebujesz ML w strumieniu | MLlib, integracja z pandas UDF |
| Zespol zna Spark/SQL | Latwiejsze wdrozenie |
| Laczenie batch + streaming | Unified API (Delta Lake, Iceberg) |
| Ekosystem Hadoop | Lepsza integracja z HDFS, Hive, HBase |

### Podsumowanie dla naszego systemu rekomendacji MovieLens:

```
Przypadek uzycia                    Lepszy wybor
─────────────────────────────────────────────────
Batch ETL (zaladuj ratings)          Spark
Trening modelu ALS                   Spark (MLlib)
Real-time trending movies            Flink (niska latencja)
Sesje uzytkownikow                   Flink (session windows)
Streaming + zapis do HDFS            Spark (natywna integracja)
Alerty o anomaliach w ocenach        Flink (CEP)
Dashboard z opoznieniem ~2s          Spark (prostsze)
```

In [ ]:
# Podsumowanie porownania w formie tabeli
import pandas as pd

comparison = pd.DataFrame({
    "Cecha": [
        "Model przetwarzania", "Typowa latencja", "Gwarancje dostarczenia",
        "State backend", "Session windows", "Late data handling",
        "Checkpointing", "API Python", "Batch processing",
        "Ekosystem ML", "Dojrzalosc", "Krzywa uczenia"
    ],
    "Apache Flink": [
        "Natywny streaming", "<100ms", "Exactly-once (Chandy-Lamport)",
        "RocksDB / Heap (TB stanu)", "Natywne", "Side output (oddzielny strumien)",
        "Asynchroniczne, inkrementalne", "PyFlink (Table + DataStream)", "Streaming jako special case",
        "Ograniczony (FlinkML exp.)", "Dojrzaly (prod od 2016)", "Stroma"
    ],
    "Spark Structured Streaming": [
        "Micro-batch (domyslnie)", "500ms - 2s", "Exactly-once (WAL + checkpoint)",
        "HDFS State Store (ograniczony)", "Brak natywnych", "Drop (odrzucenie)",
        "Synchroniczne, pelny snapshot", "PySpark (DataFrame + SQL)", "Natywny batch engine",
        "MLlib, pandas UDF", "Bardzo dojrzaly", "Lagodna (jesli znasz Spark)"
    ]
})

# Wyswietl bez obcinania
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.width', 150)
print(comparison.to_string(index=False))

## Zadanie koncowe

Zaimplementuj **real-time anomaly detection** w strumieniu ocen filmow, uzywajac zarowno Spark Structured Streaming jak i PyFlink.

### Scenariusz:
System powinien wykrywac anomalie w ocenach - np. film nagle dostaje wiele niskich ocen (atak botow?) lub jeden uzytkownik ocenia setki filmow w krotkim czasie.

### Wymagania:

**Czesc 1 - Spark Structured Streaming:**
1. Stworz strumien z file source (uzyj `generate_rating_events`)
2. Tumbling window 30s z watermarkiem 1 min
3. Wykryj filmy z >20 ocenami w oknie i srednia <2.0 (podejrzane)
4. Wykryj uzytkownikow z >10 ocenami w oknie (podejrzana aktywnosc)
5. Wyswietl alerty w konsoli

**Czesc 2 - PyFlink:**
1. Uzyj Table API z tym samym zrodlem danych
2. Session window z gap 20s per uzytkownik (wykryj sesje)
3. Wykryj sesje z >15 ocenami (anomalia)
4. Porownaj wyniki z czescia Spark

**Czesc 3 - Porownanie:**
1. Zmierz latencje obu podejsc
2. Ktore podejscie lepiej wykrywa anomalie? Dlaczego?
3. Napisz krotkie podsumowanie (3-5 zdan)

In [ ]:
# Twoje rozwiazanie:

# --- Czesc 1: Spark Structured Streaming ---


# --- Czesc 2: PyFlink ---


# --- Czesc 3: Porownanie ---


In [ ]:
# Cleanup
import shutil
shutil.rmtree(STREAM_DIR, ignore_errors=True)
shutil.rmtree(CHECKPOINT_DIR, ignore_errors=True)

spark.stop()
print("Sesje Spark i Flink zamkniete. Dane tymczasowe usuniete.")